In [6]:
# # Notebook 1: Data Preprocessing Pipeline
# ## Objective: 
# - Load and merge the 3 Amazon review CSV files
# - Clean and preprocess review text (remove noise, normalize, tokenize, etc.)
# - Handle missing values and duplicates
# - Save preprocessed data for downstream tasks (NER, sentiment analysis, etc.)


# ### 1. Import Libraries
# %%
import pandas as pd
import numpy as np
import re
import spacy
import textblob  # For spelling correction
from spacy.lang.en.stop_words import STOP_WORDS
from tqdm import tqdm  # For progress bars
import warnings
warnings.filterwarnings("ignore")

# Load spaCy's English model (for tokenization, lemmatization)
nlp = spacy.load("en_core_web_sm")

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("datafiniti/consumer-reviews-of-amazon-products")

print("Path to dataset files:", path)

In [ ]:
# ### 2. Load and Merge Datasets
# Your dataset includes 3 CSV files. We’ll load them, check for consistent columns, and merge.
# %%
import pandas as pd
import os

# Path to dataset
path = kagglehub.dataset_download("datafiniti/consumer-reviews-of-amazon-products")

# List CSV files in the folder
csv_files = [f for f in os.listdir(path) if f.endswith(".csv")]
csv_files.sort()  # optional: to ensure consistent order

# Load each CSV into a DataFrame
df1 = pd.read_csv(os.path.join(path, csv_files[0]))
df2 = pd.read_csv(os.path.join(path, csv_files[1]))
df3 = pd.read_csv(os.path.join(path, csv_files[2]))

print("Loaded files:")
print(csv_files)

In [8]:
# #### 2.1 Inspect Columns (Ensure Consistency)
# Amazon review datasets often have columns like `reviewText`, `rating`, `product_name`, etc. We need to standardize column names.
# %%
print("Columns in df1:", df1.columns.tolist())
print("\nColumns in df2:", df2.columns.tolist())
print("\nColumns in df3:", df3.columns.tolist())


Columns in df1: ['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer', 'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title', 'reviews.userCity', 'reviews.userProvince', 'reviews.username']

Columns in df2: ['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand', 'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer', 'manufacturerNumber', 'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs']

Columns in df3: ['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand', 'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer', 'manufacturerNumber', 'reviews.date', 'reviews.dateSeen', 'reviews.didPurchase', 'reviews.do

In [10]:

# #### 2.1 Standardize Columns (Critical for Merging)
# Based on your column lists, all 3 DataFrames share these key columns:
# - `name`: Product name  
# - `reviews.text`: Raw review content  
# - `reviews.rating`: Star rating (1-5)  
# - `reviews.date`: Date of the review  

# We’ll rename these to simpler labels and keep only these columns for our tasks.
# %%
def standardize_columns(df):
    """Standardize column names and keep only relevant columns."""
    # Rename columns to intuitive labels
    column_mapping = {
        "name": "product_name",          # Product name
        "reviews.text": "review_text",   # Raw review content
        "reviews.rating": "rating",      # Star rating (1-5)
        "reviews.date": "review_date"    # Date of the review
    }
    # Rename columns and keep only the mapped ones
    df_standardized = df.rename(columns=column_mapping)[column_mapping.values()]
    return df_standardized

# Apply standardization to all DataFrames
df1_clean = standardize_columns(df1)
df2_clean = standardize_columns(df2)
df3_clean = standardize_columns(df3)

# Verify column consistency after standardization
print("Standardized columns (all DataFrames):", df1_clean.columns.tolist())

Standardized columns (all DataFrames): ['product_name', 'review_text', 'rating', 'review_date']


In [11]:
# #### 2.2 Merge Datasets and Check Size
# %%
# Merge all 3 DataFrames
combined_df = pd.concat([df1_clean, df2_clean, df3_clean], ignore_index=True)
print(f"Total reviews after merging: {len(combined_df):,}")

Total reviews after merging: 67,992


In [12]:
# ### 3. Handle Missing Values and Duplicates
# Critical for reliable NLP: Remove rows with missing review text or ratings, and drop duplicate reviews.
# %%
# Check missing values in key columns
print("Missing values before cleaning:")
print(combined_df.isnull().sum())

Missing values before cleaning:
product_name    6760
review_text        1
rating            33
review_date       39
dtype: int64


In [13]:
# Drop rows with missing review text or ratings (core for NLP tasks)
combined_df = combined_df.dropna(subset=["review_text", "rating"]).reset_index(drop=True)

# Drop duplicate reviews (same product + same review text = redundant)
combined_df = combined_df.drop_duplicates(subset=["product_name", "review_text"], keep="first")

# Convert rating to numeric (in case it’s stored as string)
combined_df["rating"] = pd.to_numeric(combined_df["rating"], errors="coerce")
# Drop any remaining non-numeric ratings
combined_df = combined_df.dropna(subset=["rating"]).reset_index(drop=True)

print(f"\nTotal reviews after cleaning: {len(combined_df):,}")


Total reviews after cleaning: 64,037


In [14]:
# ### 4. Text Preprocessing Pipeline
# Clean and normalize `review_text` for downstream tasks (NER, sentiment analysis, etc.):
# 1. Remove URLs, special characters, and numbers  
# 2. Correct minor spelling errors  
# 3. Tokenize, remove stopwords (e.g., "the", "and"), and lemmatize (e.g., "running" → "run")  
# %%
def clean_text(raw_text):
    """Step 1: Remove noise (URLs, special characters, numbers)."""
    # Remove URLs (e.g., "https://example.com")
    text = re.sub(r"http\S+", "", raw_text)
    # Remove numbers and special characters (keep only letters and spaces)
    text = re.sub(r"[^a-zA-Z\s]", "", raw_text)
    # Collapse multiple spaces into one
    text = re.sub(r"\s+", " ", text).strip()
    return text

def correct_spelling(text):
    """Step 2: Correct minor spelling errors (e.g., "teh" → "the")."""
    # Limit correction to short texts to avoid slow processing
    if len(text) > 500:  # Skip long texts (speed optimization)
        return text
    return str(textblob.TextBlob(text).correct())

def preprocess_text(text):
    """Full preprocessing: clean → spell check → tokenize → lemmatize → remove stopwords."""
    # Step 1: Clean text
    cleaned = clean_text(text)
    if not cleaned:  # Skip empty texts after cleaning
        return ""
    
    # Step 2: Correct spelling (optional but improves NER accuracy)
    spelled_correct = correct_spelling(cleaned)
    
    # Step 3: Process with spaCy (lowercasing, tokenization, lemmatization)
    doc = nlp(spelled_correct.lower())  # Convert to lowercase first
    tokens = []
    for token in doc:
        # Skip stopwords (e.g., "is", "and") and punctuation
        if token.is_stop or token.is_punct:
            continue
        # Keep lemmatized form (base word)
        tokens.append(token.lemma_)
    
    # Join tokens back into a single string
    return " ".join(tokens)

In [15]:
# #### 4.1 Apply Preprocessing to All Reviews
# *Note: This may take 10-15 minutes for 30k+ reviews. Use `tqdm` to track progress.*
# %%
# Apply preprocessing with a progress bar
tqdm.pandas(desc="Preprocessing reviews")
combined_df["cleaned_review"] = combined_df["review_text"].progress_apply(preprocess_text)

# Remove rows where preprocessing resulted in empty text (rare, but possible)
combined_df = combined_df[combined_df["cleaned_review"] != ""].reset_index(drop=True)


Preprocessing reviews:  37%|███▋      | 23856/64037 [1:07:36<7:07:01,  1.57it/s] IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Preprocessing reviews: 100%|██████████| 64037/64037 [2:49:45<00:00,  6.29it/s]   


In [16]:
# ### 5. Create Sentiment Labels (for Sentiment Analysis)
# Map numeric ratings to sentiment categories:
# - Rating ≥ 4 → Positive (1)  
# - Rating ≤ 2 → Negative (0)  
# - Rating = 3 → Neutral (2)  
# %%
def get_sentiment(rating):
    if rating >= 4:
        return 1  # Positive
    elif rating <= 2:
        return 0  # Negative
    else:
        return 2  # Neutral

combined_df["sentiment_label"] = combined_df["rating"].apply(get_sentiment)

# Check distribution of sentiment labels
print("\nSentiment label distribution:")
print(combined_df["sentiment_label"].value_counts(normalize=True).round(3) * 100)


Sentiment label distribution:
sentiment_label
1    91.9
2     4.3
0     3.8
Name: proportion, dtype: float64


In [17]:
# ### 6. Save Preprocessed Data
# Save the cleaned dataset for use in Notebook 2 (Basic NLP Techniques) and beyond.
# %%
combined_df.to_csv("preprocessed_amazon_reviews.csv", index=False)
print("\nPreprocessed data saved as 'preprocessed_amazon_reviews.csv'")


Preprocessed data saved as 'preprocessed_amazon_reviews.csv'


In [18]:
# ### 7. Validate Preprocessing Quality
# Spot-check 5 random reviews to ensure cleaning worked as expected.
# %%
# Random sample of 5 reviews (raw vs. cleaned)
sample = combined_df.sample(5)[[
    "product_name", 
    "review_text", 
    "cleaned_review", 
    "rating", 
    "sentiment_label"
]]
display(sample)

,product_name,review_text,cleaned_review,rating,sentiment_label
35292,Amazon Echo Show Alexa-enabled Bluetooth Speak...,I am so happy to have the Echo Show. I have be...,happy echo able listen christmas song country ...,5.0,1
55943,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","Holds a long charge. Fast great space, no free...",fold long charge great space freezing resecting,5.0,1
53856,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...",Kindle Fire meets the needs of kids and/or adu...,kindle fire meet need kiss ardor adult,5.0,1
7971,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",This tablet is a great purchase for the price....,tablet great purchase price easy use perfect kiss,4.0,1
23816,"Echo (White),,,\r\nEcho (White),,,",Fun to play with.. Was alittle difficult to ge...,run play little difficult set,4.0,1
